The Pacman environment
----------------------
The Pacman is an agent that moves around the grid. When it encounters a cell with a breadcrumb, it "eats" it and gets a
positive reward. When all of the breadcrumbs are consumed, he wins.

The Pacman moves 1 cell for each timestep in one of 4 directions; up, down, left, right. All 4 sides of the 
environment grid contain obstacles so that the agent cannot move out of the grid. In addition there are more obstacles
inside the grid as well. If the Pacman tries to move into an obstacle, he gets a negative reward and returns back to the 
cell he moved from. If the Pacman moves into an empty cell, he gets a small negative reward. This is an incentive to get 
games completed as quickly as possible.

The whole algorithm is run from the code below.

Before running this code, check in the config.py file for the configurable parameters used in this algorithm.
This includes the number of breadcrumbs, the location of the breadcrumbs and the location of the obstacles.
The Pacman is more likely to win if there are fewer breadcrumbs in the grid.

The Pacman learns from completing a large number of episodes. If you intend to run lots of episodes, it is a 
good idea to set the flag show_step = False as step by step changes will output too many lines.

The ghost provides stochasticity in the environment. 

If the Pacman moves onto the ghost, he loses. There is a flag "Hyper.is_ghost" which when set to True ensures
there is a ghost in the environment.

In [1]:
from grid import Pacman_grid
from config import Hyper, Constants

print("\n"*10)
print("-"*100)
print("Start of environment design for Pacman")
Hyper.display()
print("-"*100)
print("The grid is updated and printed for each step. In each cell you see the following symbols:")
print("X - obstacle")
print(". - empty")
print("b - breadcrumb")
print("A - Agent (Pacman)")
print("G - Ghost")
pacman_grid = Pacman_grid()
for i in range(Hyper.total_episodes):
    pacman_grid.reset()
    done = False
    while done == False:
        if Hyper.is_ghost:
            done = pacman_grid.ghost_step(i)
        else:
            done = pacman_grid.step(i)
        pacman_grid.policy.update_epsilon()
    episodes = i + 1
    pacman_grid.print_episode_results(episodes)
    pacman_grid.save_episode_stats()

pacman_grid.print_results()
print("\nThe grid is updated and printed for each step. In each cell you see the following symbols:")
print("X - obstacle")
print(". - empty")
print("b - breadcrumb")
print("A - Agent (Pacman)")
print("G - Ghost")
print("\n"*3)  
print("-"*100)
Hyper.display()
print("End of environment design for Pacman")
print("-"*100)












----------------------------------------------------------------------------------------------------
Start of environment design for Pacman
The Hyperparameters
-------------------
Threshold for exploitation (epsilon) = 0.95
epsilon decay = 0.995
minimum value of epsilon = 0.001
learning rate (alpha) = 0.8
discount factor (gamma) = 0.51
total number of breadcrumbs 10
----------------------------------------------------------------------------------------------------
The grid is updated and printed for each step. In each cell you see the following symbols:
X - obstacle
. - empty
b - breadcrumb
A - Agent (Pacman)
G - Ghost
Completed environment after 1 episodes and 99 timesteps, total reward: -4543 with epsilon: 0.5783737835841118
You lost to the ghost!
Completed environment after 2 episodes and 43 timesteps, total reward: -1462 with epsilon: 0.4662309189661894
You lost to the ghost!
Completed environment after 3 episodes and 38 timesteps, total reward: -2088 with epsilon: 0.38

When all of the episodes are completed, look in the local images folder to view the results

The next cell is code to run for Hyperameter tuning using the Optuna library.
Please run the following command to install the Optuna library;

pip install optuna

Given the stochastic environment, I had to run 10,000 trials 4 times to get a good idea 
of what the best hyperparameters are. This takes a long time, about 3 hours.
To check that the code works, you can run fewer trials, for example 1,000.
The class libraries are the same as referenced in the previous cell.

In [3]:
import optuna
from grid import Pacman_grid
from config import Hyper, Constants

# This code uses the Optuna library to tune the hyperparameters
# Instead of producing graphs, it produces statistics


def objective(trial):
    Hyper.print_episodes = False    # Recommended do not show episode information, too much output
    Hyper.gamma = trial.suggest_float("Hyper.gamma", 0.01, 0.99)
    Hyper.alpha = trial.suggest_float("Hyper.alpha", 0.01, 0.99)
    Hyper.init_epsilon = trial.suggest_float("Hyper.init_epsilon", 0.9, 1)
    Hyper.epsilon_threshold = trial.suggest_float("Hyper.epsilon_threshold", 0.001, 0.1)
    Hyper.decay = trial.suggest_float("Hyper.decay", 0.990, 0.999)
    pacman_grid = Pacman_grid()
    for i in range(Hyper.total_episodes):
        pacman_grid.reset()
        done = False
        while done == False:
            if Hyper.is_ghost:
                done = pacman_grid.ghost_step(i)
            else:
                done = pacman_grid.step(i)
            pacman_grid.policy.update_epsilon()
        episodes = i + 1
        if Hyper.print_episodes:
            pacman_grid.print_episode_results(episodes)
        pacman_grid.save_episode_stats()

    # Average the rewards at the end to even out any outlier results caused
    # by the stochastic environment.
    sample = 100
    last_sample = -1 * sample
    reward_for_last_sample_episode = sum(pacman_grid.rewards_per_episode[last_sample:]) / sample
    return reward_for_last_sample_episode
    
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
# Note n_trials=10000. This is a lot of trials and will take a long time to run, but the results will be better.
# The important results are at the end of the printout
study.optimize(objective, n_trials=10000)
print("\n")
print("-"*100)
print("Number of finished trials: ", len(study.trials))
print("The optimal hyperpameters are as follows;")
print(study.best_params)
print(study.best_value)     

[I 2021-04-01 16:02:46,364] A new study created in memory with name: no-name-889ab2ec-6de8-4ebe-87e6-29efbbae89c5
[I 2021-04-01 16:02:47,374] Trial 0 finished with value: -175.09 and parameters: {'Hyper.gamma': 0.15616262555963364, 'Hyper.alpha': 0.9158543426419831, 'Hyper.init_epsilon': 0.9785465260795877, 'Hyper.epsilon_threshold': 0.06966007345727257, 'Hyper.decay': 0.9976583905436357}. Best is trial 0 with value: -175.09.
[I 2021-04-01 16:02:48,790] Trial 1 finished with value: -129.43 and parameters: {'Hyper.gamma': 0.7847656753730818, 'Hyper.alpha': 0.6003570414881729, 'Hyper.init_epsilon': 0.9872448490468405, 'Hyper.epsilon_threshold': 0.04849028134092385, 'Hyper.decay': 0.9909177477413627}. Best is trial 1 with value: -129.43.
[I 2021-04-01 16:02:49,820] Trial 2 finished with value: -32.33 and parameters: {'Hyper.gamma': 0.11191724226355902, 'Hyper.alpha': 0.7997231793718556, 'Hyper.init_epsilon': 0.9744286807986741, 'Hyper.epsilon_threshold': 0.002159873458454041, 'Hyper.decay